#Mastering the Story Cloze task with BERT

In this notebook we use BERT's [Sentence (and sentence-pair) classification task](https://https://github.com/graulef/bert#sentence-and-sentence-pair-classification-tasks) to classify whether given a four sentence story, a candidate ending is indeed the correct one. 

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import numpy as np

!pip install bert-tensorflow

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

from tensorflow import keras
import os
import re
import csv
import random

PATH_STORY_CLOZE_TRAINING_DATA = "http://felix.graule.ch/wp-content/uploads/2019/05/cloze_test_val_spring2016.csv"
PATH_STORY_CLOZE_TEST_DATA = "http://felix.graule.ch/wp-content/uploads/2019/06/test_for_report-stories_labels.csv"
PATH_BACKWARD_DATA = "http://felix.graule.ch/wp-content/uploads/2019/06/train_stories_backward_combined.csv"
PATH_NN_ENDING_SENT2VEC_DATA = "http://felix.graule.ch/wp-content/uploads/2019/06/train_stories_nearest_last_sent2vec_combined.csv"
PATH_NN_STORY_SENT2VEC_DATA = "http://felix.graule.ch/wp-content/uploads/2019/05/train_stories_nearest_story_sent2vec_combined.csv"
PATH_NN_ENDING_RANDOM_DATA = "http://felix.graule.ch/wp-content/uploads/2019/05/train_stories_rand_combined.csv"
PATH_NN_STORY_USC_DATA = "http://felix.graule.ch/wp-content/uploads/2019/05/train_stories_nearest_story_usc_combined.csv"
PATH_NN_STORY_USC_NLP_DATA = "http://felix.graule.ch/wp-content/uploads/2019/06/train_stories_nearest_story_usc_with_nlp_features_combined.csv"
PATH_GPT_ENDING_DATA = "http://felix.graule.ch/wp-content/uploads/2019/06/gpt_output.txt"

# Create random file name to enforce retraining of model
rand = random.randint(0, 1000)
OUTPUT_DIR = 'tmp_folder_' + str(rand)

W0606 11:15:56.090628 139692871968640 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


     |████████████████████████████████| 71kB 4.8MB/s 


#Chose Experiment and Load Data

In [3]:
# Choose which experiment should be run
EXPERIMENT = "SIMPLE" # can be "SIMPLE", "MIXED" or "GPT"

TRAIN_DATA_PATH = PATH_STORY_CLOZE_TRAINING_DATA # only needed if EXPERIMENT == "SIMPLE"

# Load all files from a directory in a DataFrame.
def load_combined_data(file):
  data_1 = {}
  data_1["label"] = []
  data_1["id_1"] = []
  data_1["id_2"] = []
  data_1["context"] = []
  data_1["ending"] = []
  
  data_2 = {}
  data_2["label"] = []
  data_2["id_1"] = []
  data_2["id_2"] = []
  data_2["context"] = []
  data_2["ending"] = []

  with open(file) as f:
    csv_reader = csv.reader(f, delimiter=',')
    line_count = 0
    for row in csv_reader:
      if line_count == 0:
        #print("Columns = " + str(row))
        line_count += 1
      else:
        line_count += 1
        
        # Create two lines from one to have same label layout as MRPC task
        seperator = ' '
        data_1["id_1"].append(row[0])
        data_1["id_2"].append(row[0] + "_end_bli")
        data_1["context"].append(str(seperator.join(row[1:5])))
        
        data_2["id_1"].append(row[0])
        data_2["id_2"].append(row[0] + "_end_bla")
        data_2["context"].append(str(seperator.join(row[1:5])))
        
        if row[7] == "1": # First ending is the correct one
          data_1["ending"].append(row[5])
          data_1["label"].append(1)
          data_2["ending"].append(row[6])
          data_2["label"].append(0)
        else: # Second ending is the correct one
          data_1["ending"].append(row[6])
          data_1["label"].append(1)
          data_2["ending"].append(row[5])
          data_2["label"].append(0) 
          
    data_df_1 = pd.DataFrame.from_dict(data_1)
    data_df_2 = pd.DataFrame.from_dict(data_2)  
    data = pd.concat([data_df_1, data_df_2])  
    return data   

def load_test_dataset():
  test = tf.keras.utils.get_file(
      fname="test", 
      origin=PATH_STORY_CLOZE_TEST_DATA)
  data_df = load_combined_data(test)
  test_df = data_df
  return test_df

def load_one_combined_training_dataset():
  # Create random file name to enforce redownloading of dataset
  rand = random.randint(0, 1000)
  file_name = "tmp_" + str(rand)
  train = tf.keras.utils.get_file(
      fname=file_name, 
      origin=TRAIN_DATA_PATH)
  data_df = load_combined_data(train)
  train_df = data_df.sample(frac=1).reset_index(drop=True)
  return train_df

def load_mixed_training_dataset():
  # TODO: Define mixture
  pass

def load_gpt_data():
  train = tf.keras.utils.get_file(
      fname="gpt", 
      origin=PATH_GPT_ENDING_DATA)


# Run data handling

if EXPERIMENT == "GPT":
  train = load_gpt_data()
elif EXPERIMENT == "MIXED":
  train = load_mixed_training_dataset()
elif EXPERIMENT == "SIMPLE":
  train = load_one_combined_training_dataset()
else:
  print("Unknonw experiment type")
  
test = load_test_dataset()

print("\nTrain data")
print(train.shape)
for i in range(5):
  print(train.iloc[i]['label'])
  print(train.iloc[i]['context'])
  print(train.iloc[i]['ending'])

print("\nTest data")
print(test.shape)
for i in range(5):
  print(test.iloc[i]['label'])
  print(test.iloc[i]['context'])
  print(test.iloc[i]['ending'])
  
mid_offset = int(test.shape[0]/2)
for i in range(5):
  print(test.iloc[i+mid_offset]['label'])
  print(test.iloc[i+mid_offset]['context'])
  print(test.iloc[i+mid_offset]['ending'])

606208/600217 [==============================] - 1s 1us/step

Train data
(3742, 5)
0
Bob had recently just purchased his dream car. The only thing he didn't like about it was the color. He took it to the paint shop to get it repainted. He waited a week and then picked it up from the shop.
Bob wished the car was the original color.
1
Sam regretted not starting his history essay earlier. He typed up his closing paragraphs. All of a sudden, his screen was covered in a large blue error message. Sam shrugged and restarted his computer.
He was disappointed in his broken computer.
1
Mim was driving down the highway in a bad snow storm. There was so much snow, Mim pulled her car over. She noticed other people were doing the same. She got an emergency blanket from the backseat, and ate a snack.
Eventually it stopped snowing.
0
Janice usually wears jeans to work every day. However, now she has been promoted to manager. She decides she needs to dress a little more formally. Janice buys a few pair

#Data Preprocessing, Model Loading and Tokenization

In [4]:
CONTEXT_COLUMN = 'context'
ENDING_COLUMN = 'ending'
LABEL_COLUMN = 'label'
label_list = [0, 1]

# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[CONTEXT_COLUMN], 
                                                                   text_b = x[ENDING_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[CONTEXT_COLUMN], 
                                                                   text_b = x[ENDING_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

Instructions for updating:
Colocations handled automatically by placer.


W0606 11:16:09.262844 139692871968640 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 11:16:11.153589 139692871968640 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 3742


...

I0606 11:16:16.543927 139692871968640 run_classifier.py:466] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0606 11:16:16.545953 139692871968640 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


I0606 11:16:16.548177 139692871968640 run_classifier.py:468] label: 1 (id = 1)


#Adjust the model

We add a single layer to adapt BERT to our classification task.

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [7]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

INFO:tensorflow:Using config: {'_model_dir': 'tmp_folder_352', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0c4050dac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0606 11:16:19.804952 139692871968640 estimator.py:201] Using config: {'_model_dir': 'tmp_folder_352', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0c4050dac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Training

In [8]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0606 11:16:21.743829 139692871968640 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 11:16:24.639855 139692871968640 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 1169 into tmp_folder_352/model.ckpt.


I0606 11:35:00.233063 139692871968640 basic_session_run_hooks.py:594] Saving checkpoints for 1169 into tmp_folder_352/model.ckpt.


INFO:tensorflow:Loss for final step: 0.00037022203.


I0606 11:35:10.229004 139692871968640 estimator.py:359] Loss for final step: 0.00037022203.


Training took time  0:18:50.410820


# Testing

In [9]:
test_input_fn = bert.run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0606 11:35:12.678927 139692871968640 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 11:35:16.116796 139692871968640 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 1169: auc = 0.81854624, eval_accuracy = 0.81854624, f1_score = 0.82173795, false_negatives = 306.0, false_positives = 373.0, global_step = 1169, loss = 1.0507787, precision = 0.80753356, recall = 0.8364511, true_negatives = 1498.0, true_positives = 1565.0


I0606 11:36:07.643722 139692871968640 estimator.py:1979] Saving dict for global step 1169: auc = 0.81854624, eval_accuracy = 0.81854624, f1_score = 0.82173795, false_negatives = 306.0, false_positives = 373.0, global_step = 1169, loss = 1.0507787, precision = 0.80753356, recall = 0.8364511, true_negatives = 1498.0, true_positives = 1565.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1169: tmp_folder_352/model.ckpt-1169


I0606 11:36:09.966222 139692871968640 estimator.py:2039] Saving 'checkpoint_path' summary for global step 1169: tmp_folder_352/model.ckpt-1169


{'auc': 0.81854624,
 'eval_accuracy': 0.81854624,
 'f1_score': 0.82173795,
 'false_negatives': 306.0,
 'false_positives': 373.0,
 'global_step': 1169,
 'loss': 1.0507787,
 'precision': 0.80753356,
 'recall': 0.8364511,
 'true_negatives': 1498.0,
 'true_positives': 1565.0}

# Prediction to get accuracy for original Story Cloze task

In [10]:
results = estimator.predict(input_fn=test_input_fn)
results = list(results)

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only differenc

# Organize test data as pairs, first version is story with correct ending and
# second has wrong ending
correct_predictions_cross_check = 0
correct_predictions_story_cloze = 0
number_weird_examples = 0
number_pairs = int(test.shape[0] / 2)

for i in range(number_pairs):
  # Get pairwise data and predictions
  right_story = test.iloc[i]
  prob_right_ending = softmax(results[i]['probabilities'])
  wrong_story = test.iloc[i+number_pairs]
  prob_wrong_ending = softmax(results[i+number_pairs]['probabilities'])
  
  # Check if predictions are true vice-versa (cross check) or true one-way (story cloze)
  is_correct_prediction_cross_check = (prob_right_ending[0] < prob_right_ending[1] and
                                       prob_wrong_ending[0] > prob_wrong_ending[1])
  is_correct_prediction_story_cloze = prob_right_ending[1] > prob_wrong_ending[1]
  
  
  # Increase result counters
  if is_correct_prediction_cross_check:
    correct_predictions_cross_check += 1
  if is_correct_prediction_story_cloze:
    correct_predictions_story_cloze += 1
  predictions_too_similar = abs(prob_right_ending[1] - prob_wrong_ending[1]) < 0.01
  
  is_weird_example = False
  if not is_correct_prediction_cross_check and is_correct_prediction_story_cloze and predictions_too_similar:
    number_weird_examples += 1
    is_weird_example = True

  # Print examples   
  if i < 20:
    if is_weird_example: 
      print("This is a weird example!")
      
      # Unindent this block if not only weird examples should be printed!
      print("\nStory Pair number " + str(i))
      print("Correct Story context: " + right_story['context'])
      print("Correct Story ending: " + right_story['ending'])
      print("Predictions for Correct Story (label 0 / label 1): " + str(prob_right_ending))

      print("Wrong Story context: " + wrong_story['context'])
      print("Wrong Story ending: " + wrong_story['ending'])
      print("Predictions for Wrong Story (label 0 / label 1): " + str(prob_wrong_ending))

      print("Is correctly predicted according to Cross Check task: " + str(is_correct_prediction_cross_check))
      print("Is correctly predicted according to Story Cloze task: " + str(is_correct_prediction_story_cloze))
    

accuracy_cross_check = correct_predictions_cross_check/number_pairs
print("Accuracy for Cross Check is: " + str(accuracy_cross_check))
accuracy_story_cloze = correct_predictions_story_cloze/number_pairs
print("Accuracy for Story Cloze task is: " + str(accuracy_story_cloze))
ratio_weird_examples = number_weird_examples/number_pairs
print("Ratio of weird examples is: " + str(ratio_weird_examples))

INFO:tensorflow:Calling model_fn.


I0606 11:36:11.938983 139692871968640 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 11:36:15.147713 139692871968640 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


I0606 11:36:16.746778 139692871968640 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0606 11:36:16.826997 139692871968640 session_manager.py:493] Done running local_init_op.


This is a weird example!

Story Pair number 4
Correct Story context: John was writing lyrics for his new album. He started experiencing writer's block. He tried to force himself to write but it wouldn't do anything. He took a walk, hung out with some friends, and looked at nature.
Correct Story ending: He felt inspiration and then went back home to write.
Predictions for Correct Story (label 0 / label 1): [2.050722e-04 9.997949e-01]
Wrong Story context: John was writing lyrics for his new album. He started experiencing writer's block. He tried to force himself to write but it wouldn't do anything. He took a walk, hung out with some friends, and looked at nature.
Wrong Story ending: John then got an idea for his painting.
Predictions for Wrong Story (label 0 / label 1): [0.00280675 0.9971933 ]
Is correctly predicted according to Cross Check task: False
Is correctly predicted according to Story Cloze task: True
This is a weird example!

Story Pair number 13
Correct Story context: Aaron's

In [0]:
# Modified by Felix Graule, 2019

# Original version by Google Inc.

# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.